<a href="https://colab.research.google.com/github/Hussy24/Predict-Price-for-Gifts/blob/main/hasnain_sajid(27199).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
# Read the CSV file
df = pd.read_csv('dataset.csv')
df.head


<bound method NDFrame.head of         gift_id  gift_type  gift_category  gift_cluster  \
0      GF_11156         61            534          3942   
1      GF_11157         61            534          3942   
2      GF_15689        584            262             0   
3      GF_11155         61            534          3942   
4      GF_11158         61            534          3942   
...         ...        ...            ...           ...   
20274  GF_10269        105            704          6448   
20275   GF_5854       1220            526           817   
20276    GF_563        509            705           821   
20277   GF_9107        682            213          5620   
20278   GF_5683       1061            212          4987   

                  instock_date        stock_update_date  lsg_1  lsg_2  lsg_3  \
0      2014-02-21 05:07:06.000  2016-11-09 15:49:51.000   3377   5221    504   
1      2014-02-21 06:07:06.000  2016-11-11 13:49:51.000   3377   5221    504   
2      2014-02-21 09:

In [18]:
# dropping null rows
df = df.dropna()

# Extract the input and target variables for predicting files
X = df[['volumes', 'gift_type', 'gift_category', 'gift_cluster', 'lsg_1', 'lsg_2', 'lsg_3','lsg_4', 'lsg_5', 'lsg_6', 'is_discounted']]
Y = df['price']

In [ ]:
# Split the data into training and test sets
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Create a linear regression model
model = LinearRegression()

# Fit the model to the training data
model.fit(x_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Print the predicted values
print("Predicted values:", y_pred)